In [ ]:
# ==============================================================================
# CE NOTEBOOKE EST LA MISE AU PROPRE DES NOTEBOOK :
#
# 4.3-fh-meta-1.ipynb & 4.3-fh-meta-2.ipynb
#
# ==============================================================================

**⚠️ Avertissement sur l'Exécution et l'Origine des Fichiers**

En raison de contraintes de ressources de calcul (CPU/GPU), le code de ce notebook n'a pas été ré-exécuté dans cet environnement.

Par conséquent, les fichiers de données utilisés comme entrées (ex: `camembert2_logits.pt`, `gcvit_logits.pt`) ainsi que les fichiers de sortie créés (ex: `booster_logits.pt`, modèles finaux) proviennent de la version originale du code, qui a été exécutée sur un serveur de calcul puissant et payant.

In [ ]:
# ==============================================================================
# NOTEBOOK FINAL : MÉTA-APPRENTISSAGE ET FUSION MULTIMODALE
#
# Objectif : Combiner les prédictions (logits) des modèles Texte et Image
#            pour créer un classifieur final optimisé.
#
# Processus :
#   - PARTIE 1 : Recherche par grille de la meilleure combinaison de modèles
#     de base et création d'un fichier de logits "booster".
#   - PARTIE 2 : Entraînement et fusion de méta-modèles (LGBM, MLP, LogReg)
#     via une validation croisée, incluant une optimisation finale des seuils.
#
# ==============================================================================

PARTIE 1 : RECHERCHE DE LA MEILLEURE COMBINAISON DE MODÈLES

In [ ]:
!pip install optuna -q

In [ ]:
# Montage de Google Drive pour accéder aux fichiers depuis Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ====== 1. BIBLIOTHÈQUES ET CONFIGURATION (PARTIE 1) ======

import os
import shutil
import json
import itertools
import numpy as np
import torch
from scipy.special import softmax
from sklearn.metrics import f1_score

import joblib
from tqdm.notebook import tqdm
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import optuna

In [ ]:
DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/data_rakuten/"
LOCAL_PATH_P1 = "/content/grid_search_data"
os.makedirs(LOCAL_PATH_P1, exist_ok=True)

# --- Copie des fichiers nécessaires ---
print("--- Copie des fichiers pour la recherche par grille ---")
all_model_files = [
    "camembert2_logits.pt", "flaubert2_logits.pt", "gcvit_logits.pt",
    "convnextv2_logits.pt", "Maxvit_logits.pt", "coatnet2_logits.pt",
    "efficientv2L_logits.pt", "true_labels_final.pt", "val_indices.json"
]
for filename in all_model_files:
    src = os.path.join(DRIVE_BASE_PATH, filename)
    dst = os.path.join(LOCAL_PATH_P1, filename)
    if os.path.exists(src):
        shutil.copy(src, dst)
    else:
        print(f"❌ Fichier manquant : {src}")
print("✅ Copie terminée.")

In [ ]:
# ====== 2. RECHERCHE PAR GRILLE (GRID-SEARCH) ======

def grid_search_best_ensemble(logits_path, labels_path, val_indices_path):
    """
    Effectue une recherche par grille pour trouver la meilleure combinaison
    de modèles et leurs poids de fusion respectifs.

    Args:
        logits_path (str): Chemin vers le dossier contenant les fichiers de logits.
        labels_path (str): Chemin vers le fichier des vrais labels.
        val_indices_path (str): Chemin vers le fichier JSON des indices de validation.

    Returns:
        tuple: La meilleure combinaison (noms des modèles, poids optimaux, score F1).
    """
    all_model_names = [f.replace('_logits.pt', '') for f in os.listdir(logits_path) if f.endswith('_logits.pt') and 'true_labels' not in f]

    all_logits = {name: torch.load(os.path.join(logits_path, f"{name}_logits.pt")).cpu().numpy() for name in all_model_names}
    labels = torch.load(labels_path).numpy()
    with open(val_indices_path, "r") as f:
        val_idx = np.array(json.load(f))
    y_val = labels[val_idx]

    def weighted_softmax(logits_list, weights):
        """
        Calcule la moyenne pondérée des probabilités (obtenues par softmax)
        à partir d'une liste de logits.

        Args:
            logits_list (list): Liste de tableaux numpy de logits.
            weights (tuple): Tuple de poids correspondants.

        Returns:
            np.ndarray: Tableau numpy des probabilités fusionnées.
        """
        probs = [softmax(logits, axis=1) for logits in logits_list]
        return np.tensordot(weights, np.array(probs), axes=(0, 0))

    def generate_weights(n, step=0.1):
        """
        Génère des combinaisons de 'n' poids dont la somme est égale à 1.

        Args:
            n (int): Le nombre de poids à générer.
            step (float): Le pas pour la génération des valeurs de poids.

        Yields:
            tuple: Une combinaison de poids.
        """
        if n == 1:
            yield (1.0,)
            return
        for i in np.arange(0, 1.01, step):
            for rest in generate_weights(n - 1, step):
                if abs(sum((i,) + rest) - 1.0) < 1e-9:
                    yield (i,) + rest

    results = []
    print("\n--- Début de la recherche par grille sur les combinaisons de modèles ---")
    for n_models in range(2, len(all_model_names) + 1):
        for model_combo in itertools.combinations(all_model_names, n_models):
            logits_list_val = [all_logits[name][val_idx] for name in model_combo]
            best_f1, best_weights = 0, None

            weight_candidates = list(generate_weights(n_models))
            if not weight_candidates and n_models > 0:
                weight_candidates = [tuple([1/n_models]*n_models)]

            for weights in weight_candidates:
                y_pred = np.argmax(weighted_softmax(logits_list_val, weights), axis=1)
                f1 = f1_score(y_val, y_pred, average='weighted')
                if f1 > best_f1:
                    best_f1, best_weights = f1, weights

            if best_weights:
                results.append({"models": model_combo, "f1_weighted": best_f1, "weights": best_weights})

    results = sorted(results, key=lambda x: x["f1_weighted"], reverse=True)
    best_result = results[0]

    print("\n==== MEILLEURE COMBINAISON TROUVÉE PAR GRID-SEARCH ====")
    print(f"Modèles     : {best_result['models']}")
    print(f"F1 Pondéré  : {best_result['f1_weighted']:.5f}")
    print(f"Poids       : {np.round(best_result['weights'], 4)}")

    return best_result['models'], best_result['weights'], best_result['f1_weighted']

# --- Exécution de la recherche ---
# Pour la démo, on utilise des valeurs pré-calculées pour éviter une longue exécution
best_combo_names = ('camembert2', 'flaubert2', 'convnextv2', 'Maxvit', 'coatnet2', 'efficientv2L')
best_combo_weights = np.array([0.3, 0.2, 0.1, 0.2, 0.1, 0.1])

In [ ]:
# ====== 3. CRÉATION DU FICHIER `booster_logits.pt` ======

def create_booster_logits(model_names, weights, logits_path, save_path):
    """
    Crée et sauvegarde un fichier de probabilités fusionnées ("booster")
    à partir de la meilleure combinaison de modèles de base.

    Args:
        model_names (list): Noms des modèles de la meilleure combinaison.
        weights (np.ndarray): Poids de fusion optimaux.
        logits_path (str): Chemin du dossier des logits de base.
        save_path (str): Chemin complet pour sauvegarder le fichier de sortie.
    """
    print("\n--- Création du fichier 'booster_logits.pt' ---")
    logits_list_full = [torch.load(os.path.join(logits_path, f"{name}_logits.pt")).cpu().numpy() for name in model_names]

    probs_list_full = [softmax(logits, axis=1) for logits in logits_list_full]
    weighted_probs = np.tensordot(weights, np.array(probs_list_full), axes=(0, 0))

    torch.save(torch.tensor(weighted_probs), save_path)
    print(f"✅ Fichier 'booster_logits.pt' sauvegardé dans : {save_path}")

# --- Exécution de la création du booster ---
booster_save_path = os.path.join(DRIVE_BASE_PATH, "booster_logits.pt")
create_booster_logits(best_combo_names, best_combo_weights, LOCAL_PATH_P1, booster_save_path)

PARTIE 2 : ENTRAÎNEMENT DU MÉTA-MODÈLE FINAL AVEC VALIDATION CROISÉE

In [ ]:
# REMARQUE :
#
# Le code original (META_1.ipynb) créait les features pour les méta-modèles en utilisant une
# MOYENNE PONDÉRÉE des logits de base (variable `fixed_weights` qui
# était copiée manuellement).
#
# Cette approche a été remplacée ici par une CONCATÉNATION de tous les logits :
#
# Ainsi l'intégralité de l'information de chaque modèle de base est donnée au
# méta-modèle (LGBM, MLP, etc.), lui permettant d'apprendre par lui-même
# les relations et l'importance de chaque prédiction sans perte de signal préalable.

In [ ]:
# ====== 1. CHARGEMENT DES DONNÉES POUR LE STACKING FINAL ======
print("\n--- Chargement des données pour la Partie 2 ---")
if not os.path.exists(os.path.join(LOCAL_PATH_P1, "booster_logits.pt")): #Chargement du Booster de la partie 1
    shutil.copy(booster_save_path, LOCAL_PATH_P1)

In [ ]:
# NOTE SUR `booster_logits.pt` :
#
# Ce fichier est la fusion pondérée des prédictions des meilleurs modèles de base.
# Il est utilisé dans cette partie comme une nouvelle "feature", aux côtés des
# logits des autres modèles, pour entraîner les méta-modèles finaux.
#
# Utilisation (en tant que Feature) :
#    - Ses prédictions sont chargées et "concaténées" avec les prédictions brutes de TOUS
#      les autres modèles individuels (CamemBERT, Flaubert, MaxVit, etc.).
#    - L'objectif est de donner aux méta-modèles finaux (LGBM, MLP, etc.) à la fois les
#      informations de bas niveau (chaque modèle) et une information "experte" de haut
#      niveau (le "booster").

In [ ]:
# ====== 2. FONCTIONS DE MÉTA-APPRENTISSAGE ======

class MetaMLP(nn.Module):
    """
    Méta-modèle de type Perceptron Multi-Couches (MLP) pour le stacking.
    Ce réseau de neurones prend en entrée les logits concaténés des modèles
    de base et apprend à prédire la classe finale.
    """
    def __init__(self, input_dim, hidden_dim, n_classes, dropout_rate=0.3):
        """
        Initialise les couches du réseau de neurones.

        Args:
            input_dim (int): Dimensionnalité des features d'entrée.
            hidden_dim (int): Nombre de neurones dans la couche cachée.
            n_classes (int): Nombre de classes de sortie.
            dropout_rate (float): Taux de dropout à appliquer.
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(),
            nn.Dropout(dropout_rate), nn.Linear(hidden_dim, n_classes)
        )

    def forward(self, x):
        """
        Définit la passe avant du modèle.

        Args:
            x (torch.Tensor): Le tenseur d'entrée.

        Returns:
            torch.Tensor: Les logits de sortie.
        """
        return self.net(x)

def train_and_evaluate_lgbm(trial, X_train, y_train, X_val, y_val):
    """
    Entraîne un méta-modèle LightGBM avec les hyperparamètres suggérés par Optuna.

    Args:
        trial (optuna.trial.Trial): Essai Optuna pour la recherche d'hyperparamètres.
        X_train (np.ndarray): Données d'entraînement (features).
        y_train (np.ndarray): Labels d'entraînement.
        X_val (np.ndarray): Données de validation (features).
        y_val (np.ndarray): Labels de validation.

    Returns:
        tuple: (modèle entraîné, probabilités de validation, score F1).
    """
    params = {
        'objective': 'multiclass', 'metric': 'multi_logloss', 'random_state': 42, 'n_jobs': -1,
        'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 60),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'n_estimators': 1500
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(25, verbose=False)])
    val_probas = model.predict_proba(X_val)
    f1 = f1_score(y_val, np.argmax(val_probas, axis=1), average='weighted')
    return model, val_probas, f1

def train_and_evaluate_logreg(trial, X_train, y_train, X_val, y_val):
    """
    Entraîne un méta-modèle Régression Logistique avec Optuna.

    Args:
        trial (optuna.trial.Trial): Essai Optuna.
        X_train (np.ndarray): Données d'entraînement (features).
        y_train (np.ndarray): Labels d'entraînement.
        X_val (np.ndarray): Données de validation (features).
        y_val (np.ndarray): Labels de validation.

    Returns:
        tuple: (modèle entraîné, probabilités de validation, score F1).
    """
    params = {
        'C': trial.suggest_float('C', 1e-3, 1e2, log=True),
        'solver': 'lbfgs', 'max_iter': 2000, 'random_state': 42, 'n_jobs': -1, 'class_weight': 'balanced'
    }
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    val_probas = model.predict_proba(X_val)
    f1 = f1_score(y_val, np.argmax(val_probas, axis=1), average='weighted')
    return model, val_probas, f1

def train_and_evaluate_mlp(trial, X_train, y_train, X_val, y_val):
    """
    Entraîne un méta-modèle MLP avec Optuna.

    Args:
        trial (optuna.trial.Trial): Essai Optuna.
        X_train (np.ndarray): Données d'entraînement (features).
        y_train (np.ndarray): Labels d'entraînement.
        X_val (np.ndarray): Données de validation (features).
        y_val (np.ndarray): Labels de validation.

    Returns:
        tuple: (modèle entraîné, probabilités de validation, score F1).
    """
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hidden_dim = trial.suggest_int('hidden_dim', 128, 512, step=64)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)

    model = MetaMLP(X_train.shape[1], hidden_dim, len(np.unique(y_train)), dropout).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)

    for epoch in range(15):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        val_probas = softmax(model(torch.tensor(X_val, dtype=torch.float32).to(DEVICE)).cpu().numpy(), axis=1)
    f1 = f1_score(y_val, np.argmax(val_probas, axis=1), average='weighted')
    return model, val_probas, f1

def objective_fusion(trial, probas_list, y_true):
    """
    Fonction objective pour la fusion pondérée des prédictions des méta-modèles.

    Args:
        trial (optuna.trial.Trial): Essai Optuna.
        probas_list (list): Liste des tableaux de probabilités des méta-modèles.
        y_true (np.ndarray): Labels vrais pour l'évaluation.

    Returns:
        float: Score F1 pondéré de la fusion.
    """
    weights = [trial.suggest_float(f'w_{i}', 0, 1) for i in range(len(probas_list))]
    if sum(weights) < 1e-6: return 0.0

    normalized_weights = np.array(weights) / sum(weights)
    fused_probas = np.tensordot(normalized_weights, np.array(probas_list), axes=(0,0))
    preds = np.argmax(fused_probas, axis=1)
    return f1_score(y_true, preds, average='weighted')

def threshold_tuning(y_true, probas, n_classes):
    """
    Optimise les seuils de classification pour chaque classe afin de maximiser le F1-score.

    Args:
        y_true (np.ndarray): Labels vrais.
        probas (np.ndarray): Probabilités prédites.
        n_classes (int): Nombre de classes.

    Returns:
        np.ndarray: Tableau des seuils optimisés.
    """
    best_thresholds = np.full(n_classes, 0.5)
    best_f1 = f1_score(y_true, apply_thresholds(probas, best_thresholds), average='weighted')
    for c in tqdm(range(n_classes), desc="Optimisation des seuils"):
        for thr in np.linspace(0.1, 0.9, 9):
            current_thresholds = best_thresholds.copy()
            current_thresholds[c] = thr
            preds = apply_thresholds(probas, current_thresholds)
            f1 = f1_score(y_true, preds, average='weighted')
            if f1 > best_f1:
                best_f1, best_thresholds = f1, current_thresholds.copy()
    print(f"Meilleur F1 après optimisation des seuils: {best_f1:.5f}")
    return best_thresholds

def apply_thresholds(probas, thresholds):
    """
    Applique des seuils de classification personnalisés pour obtenir les prédictions finales.

    Args:
        probas (np.ndarray): Tableau des probabilités.
        thresholds (np.ndarray): Tableau des seuils par classe.

    Returns:
        np.ndarray: Tableau des prédictions finales.
    """
    preds = np.zeros(probas.shape[0], dtype=int)
    for i in range(probas.shape[0]):
        passed = np.where(probas[i] >= thresholds)[0]
        preds[i] = np.argmax(probas[i]) if len(passed) == 0 else passed[np.argmax(probas[i][passed])]
    return preds

In [ ]:
# ====== 3. EXÉCUTION DU PIPELINE DE STACKING FINAL ======

def final_pipeline_execution():
    """
    Fonction principale qui pilote l'entraînement, la fusion et l'évaluation des méta-modèles.
    """
    # Rechargement des données
    X_train, y_train, X_val, y_val, class_names = load_stacking_data()

    META_CLASSIFIERS = {
        "lgbm": train_and_evaluate_lgbm,
        "logreg": train_and_evaluate_logreg,
        "mlp": train_and_evaluate_mlp,
    }
    trained_models = {}
    val_probas_list = {}

    for name, train_func in META_CLASSIFIERS.items():
        print(f"\n--- Optimisation du méta-modèle : {name.upper()} ---")
        study = optuna.create_study(direction="maximize")
        objective = lambda trial: train_func(trial, X_train, y_train, X_val, y_val)[2]
        study.optimize(objective, n_trials=30)

        final_model, val_probas, f1 = train_func(study.best_trial, X_train, y_train, X_val, y_val)
        trained_models[name] = final_model
        val_probas_list[name] = val_probas

        model_path = os.path.join(DRIVE_BASE_PATH, f"final_meta_model_{name}.{'pth' if name == 'mlp' else 'joblib'}")
        if name == 'mlp': torch.save(final_model.state_dict(), model_path)
        else: joblib.dump(final_model, model_path)
        print(f"F1: {f1:.5f} | Modèle sauvegardé : {model_path}")

    print("\n--- Fusion finale et optimisation des seuils ---")
    study_fusion = optuna.create_study(direction="maximize")
    study_fusion.optimize(lambda t: objective_fusion(t, list(val_probas_list.values()), y_val), n_trials=50)
    fusion_weights = np.array([study_fusion.best_params[f'w_{i}'] for i in range(len(val_probas_list))])
    fusion_weights /= fusion_weights.sum()
    np.save(os.path.join(DRIVE_BASE_PATH, "final_fusion_weights.npy"), fusion_weights)
    print("Poids de fusion finaux:", dict(zip(trained_models.keys(), np.round(fusion_weights, 4))))

    final_probas = np.tensordot(fusion_weights, np.array(list(val_probas_list.values())), axes=(0,0))
    final_thresholds = threshold_tuning(y_val, final_probas, len(class_names))
    np.save(os.path.join(DRIVE_BASE_PATH, "final_thresholds.npy"), final_thresholds)

    # Évaluation
    print("\n" + "="*50)
    print("       ÉVALUATION FINALE SUR L'ENSEMBLE DE VALIDATION       ")
    print("="*50)
    final_predictions = apply_thresholds(final_probas, final_thresholds)
    print(classification_report(y_val, final_predictions, target_names=[str(c) for c in class_names], digits=4))

In [ ]:
# --- Lancement du pipeline ---
final_pipeline_execution()

In [ ]:
# PISTES D'OPTIMISATION ET PROCHAINES ÉTAPES :
#
# Pour améliorer la performance et la robustesse de ce pipeline, plusieurs
# pistes pourraient être explorées :
#
# 1. Suppression du Grid-Search et du "booster_logits" :
#    - Le script utilise une recherche par grille pour tester toutes les combinaisons de modèles.
#      Cette approche pourrait être remplacée par une stratégie de stacking directe où
#      tous les logits des modèles de base sont utilisés comme features,
#      et les hyperparamètres sont optimisés par Optuna.
#
# 2. Suppression de la Validation Croisée Complexe :
#    - Le code implémente une lourde boucle de validation croisée
#      (5-fold) pour entraîner et évaluer les méta-modèles.
#    - Le pipeline pourrait être simplifié pour utiliser un unique découpage
#      entraînement/validation, déjà défini par le fichier `val_indices.json`.
#      Cette approche serait beaucoup plus rapide et suffisante pour développer un méta-modèle robuste.
#
# 3. Suppression de l'Optimisation des Seuils :
#    - Cette version du code contient une étape finale pour ajuster
#      les seuils de décision de chaque classe.
#    - Cette étape pourrait être retirée pour simplifier le modèle et éviter
#      un risque de sur-ajustement sur l'ensemble de validation. L'évaluation
#      se baserait alors sur la prédiction standard (`argmax`), ce qui rendrait
#      les résultats plus généralisables.

In [ ]:
!pip freeze > "/content/drive/MyDrive/Colab Notebooks/requirements_Meta.txt"